[Shopee Code League 2021 - Multi-Channel Contacts](https://www.kaggle.com/c/shopee-code-league-2021)

In [41]:
import json
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [42]:
input_tag = "P4"
raw_df = pd.read_json('./{}/contacts.json'.format(input_tag))

In [43]:
df = raw_df.replace('', np.NaN)

In [44]:
df

,Id,Email,Phone,Contacts,OrderId
0,0,gkzAbIy@qq.com,NaN,1,NaN
1,1,NaN,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov
2,2,NaN,9125983679,0,NaN
3,3,mdllpYmE@gmail.com,NaN,0,bHquEnCbbsGLqllwryxPsNOxa
4,4,NaN,300364407,2,NaN
...,...,...,...,...,...
499995,499995,NaN,10072395382,2,whJlUOGNqjxCRzmIRdURQvlNv
499996,499996,NaN,571709661031,4,JqIXOONvrwvJxZqNxCYHqnrKS
499997,499997,NaN,4541459979,2,beXCZSzcHaBwAYoDcpQqjuAFO
499998,499998,RzSDsyH@hotmail.com,98947185431,1,ehjeFACGiwrERQxbziMxwOWku


In [45]:
%%time

email_group = df.groupby('Email').Id.agg(lambda x: set(x))
phone_group = df.groupby('Phone').Id.agg(lambda x: set(x))
order_group = df.groupby('OrderId').Id.agg(lambda x: set(x))

d = {i: set() for i in df.Id}

for ids in email_group:
    for id in ids:
        d[id] |= set(ids)
for ids in phone_group:
    for id in ids:
        d[id] |= set(ids)
for ids in order_group:
    for id in ids:
        d[id] |= set(ids)

for i in tqdm(range(3)):
    for id, ids in d.items():
        for id_ in list(ids):
            d[id] |= d[id_]

  0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 17.3 s, sys: 1.41 s, total: 18.7 s
Wall time: 18.9 s


In [46]:
id_to_contact = df.set_index('Id').Contacts.to_dict()

def get_sum_contact(ids_set):
    return sum([id_to_contact[id] for id in ids_set])

In [47]:
df['set'] = df.Id.apply(lambda x: d[x])
df['trace'] = df.set.apply(lambda x: '-'.join(map(str, sorted(list(x)))))
df['n_con'] = df.set.apply(lambda x: str(get_sum_contact(x)))
df['out'] = df.trace + ', ' + df.n_con
result_out = df[['Id', 'out']]
result_out.columns = ['ticket_id', 'ticket_trace/contact']

In [48]:
result_out.head(20)

,ticket_id,ticket_trace/contact
0,0,"0, 1"
1,1,"1-2458-98519-115061-140081-165605-476346, 12"
2,2,"2-159312-322639-348955, 4"
3,3,"3, 0"
4,4,"4, 2"
5,5,5-50-212533-215197-226720-383605-404324-458692...
6,6,"6-38-32871-142067-236367, 13"
7,7,"7, 1"
8,8,"8-183160-406623, 5"
9,9,"9-13-16708-33415-343161-417916-468927-484896, 8"


In [49]:
result_out.to_csv('result_out-{}.csv'.format(input_tag), index=False)